In [418]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [835]:
data_X = np.zeros((10000,2))
data_y = np.zeros((10000,1))
import random

for i in range(data_X.shape[0]):
    data_X[i,0] = random.uniform(0, 1)
    data_X[i,1] = random.uniform(0, 1)
    if(data_X[i,0] >= 0.5 and data_X[i,1]>=0.5):
        data_y[i] = 1
    else:
        data_y[i] = 0

In [836]:
import numpy as np
# Parameters
learning_rate = 0.01
num_steps = 500
batch_size = 200
display_step = 100

# Network Parameters
n_hidden_1 = 1 #1st layer number of neurons
#n_hidden_2 = 15 #2nd layer number of neurons
num_input = 2 # MNIST data input (img shape: 28*28)
num_classes = 1 # MNIST total classes (0-9 digits)

# initialize intervals for via
# [0,1] since sigmoid is used
input_interval = np.zeros((2,num_input))
input_interval[1,:] = input_interval[1,:] + 1
"""
input_interval = []
for i in range(num_input):
    input_interval.append([(0,255)])
input_interval = np.array(input_interval)
"""

h1_interval = np.zeros((2,n_hidden_1))
h1_interval[1,:] = h1_interval[1,:] + 1
"""
for i in range(n_hidden_1):
    h1_interval.append([0,1])
h1_interval = np.array(h1_interval)  
"""
h2_interval = np.zeros((2,n_hidden_2))
h2_interval[1,:] = h2_interval[1,:] + 1
"""
h2_interval = []
for i in range(n_hidden_2):
    h2_interval.append([(0,1)])
h2_interval = np.array(h2_interval)
"""

out_interval = np.zeros((2,num_classes))
out_interval[1,:] = out_interval[1,:] + 1
"""
out_interval = []
for i in range(num_classes):
    out_interval.append([(0,1)])
out_interval = np.array(out_interval)
"""

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [837]:
input_interval

array([[0., 0.],
       [1., 1.]])

In [838]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    #'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    #'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [839]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.sigmoid(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    # Hidden fully connected layer with 256 neurons
    #layer_2 = tf.sigmoid(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    # Output fully connected layer with a neuron for each class
    out_layer = tf.sigmoid(tf.matmul(layer_1, weights['out']) + biases['out'])
    return out_layer

In [840]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
#loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
loss = -(Y * tf.log(logits+ 1e-12) + (1 - Y) * tf.log( 1 - logits + 1e-12))
loss_op = tf.reduce_mean(tf.reduce_sum(loss, reduction_indices=[1]))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
#correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
correct_pred = tf.equal(tf.greater_equal(logits, 0.5), tf.greater_equal(Y, 0.5))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [841]:
import random
random.sample(range(30), 4)

[16, 13, 14, 9]

In [842]:
def next_batch(batch_size):
    index = random.sample(range(data_X.shape[0]), batch_size)
    return data_X[index],data_y[index]

In [843]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    logit,f_accuracy,f_weights,f_biases = sess.run([logits,accuracy,weights,biases], feed_dict={X: data_X,Y: data_y})
    print("Testing Accuracy:", f_accuracy)
    
    print(data_y)

Step 1, Minibatch Loss= 1.4192, Training Accuracy= 0.250
Step 100, Minibatch Loss= 0.7360, Training Accuracy= 0.295
Step 200, Minibatch Loss= 0.5726, Training Accuracy= 0.755
Step 300, Minibatch Loss= 0.5315, Training Accuracy= 0.780
Step 400, Minibatch Loss= 0.4476, Training Accuracy= 0.825
Step 500, Minibatch Loss= 0.4637, Training Accuracy= 0.750
Optimization Finished!
Testing Accuracy: 0.7556
[[0.29510498]
 [0.15732087]
 [0.21682264]
 ...
 [0.23624547]
 [0.316915  ]
 [0.29958618]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [0.]]


In [856]:
print(np.max(logit))

0.399206


In [844]:
import math

def sigmoid(x):
    if(x>10):
        return 1
    if(x<-30):
        return 1e-12
    return 1 / (1 + math.exp(-x))

In [845]:
import numpy as np
h1 = np.array(f_weights['h1'])
#h2 = np.array(f_weights['h2'])
out_w = np.array(f_weights['out'])
b1 = np.array(f_biases['b1'])
#b2 = np.array(f_biases['b2'])
out_b = np.array(f_biases['out'])

In [846]:
h1.shape[1]

1

In [847]:
from scipy.optimize import minimize
def objective(x,w):
    w = np.array(w)
    b = w[-1]
    w = w[:-1]
    k = sigmoid(np.sum(x*w) + b)
    return np.sum(k)
def objective_max(x,w):
    w = np.array(w)
    b = w[-1]
    w = w[:-1]
    k = sigmoid(np.sum(x*w) + b)
    return -np.sum(k)

In [848]:
# forward phase for via
def forward():
    for i in range(h1.shape[1]):
        w = h1[:,i]
        #w = np.hstack((h1[:,i],b1[i]))
        w = tuple(np.hstack((w,b1[i])))
        bnds = []
        for j in range(input_interval.shape[1]):
            bnds.append((input_interval[0,j],input_interval[1,j]))
        x0 = (0,0)
        bnds = np.array(bnds)
        bnds = tuple(bnds)
        print(w)
        sol = minimize (objective,x0,(w,),bounds = bnds)
        h1_interval[0,i] = sol['fun']
        sol = minimize(objective_max,x0,(w,),bounds = bnds)
        h1_interval[1,i] = abs(sol['fun'])
        
        
    # out layer
    for i in range(out_w.shape[1]):
        w = out_w[:,i]
        #w = np.hstack((h1[:,i],b1[i]))
        w = tuple(np.hstack((w,out_b[i])))
        bnds = []
        for j in range(h1_interval.shape[1]):
            bnds.append((h1_interval[0,j],h1_interval[1,j]))
        x0 = np.zeros((h1_interval.shape[1]))
        bnds = np.array(bnds)
        bnds = tuple(bnds)
        sol = minimize (objective,x0,(w,),bounds = bnds)
        out_interval[0,i] = sol['fun']
        sol = minimize(objective_max,x0,(w,),bounds = bnds)
        out_interval[1,i] = abs(sol['fun'])
    """
    for i in range(h2.shape[1]):
        max_interval_limit = h2[:,i]*h1_interval[1,:]
        min_interval_limit = h2[:,i]*h1_interval[0,:]
        h2_interval[0,i] = sigmoid(np.sum(min_interval_limit)+b2[i])
        h2_interval[1,i] = sigmoid(np.sum(max_interval_limit)+b2[i])
    
    for i in range(out_w.shape[1]):
        max_interval_limit = out_w[:,i]*h1_interval[1,:]
        min_interval_limit = out_w[:,i]*h1_interval[0,:]
        if(max_interval_limit<min_interval_limit):
            temp = min_interval_limit
            max_interval_limit = min_interval_limit
            min_interval_limit = temp
        out_interval[0,i] = sigmoid(np.sum(min_interval_limit) + out_b[i])
        out_interval[1,i] = sigmoid(np.sum(max_interval_limit) + out_b[i])
    """
    

In [849]:
forward()

(-2.5877903, -2.3961053, 2.5599093)


In [850]:
out_interval

array([[0.13674724],
       [0.40090991]])

In [851]:
def bound(weights,interval,bias,mode):
    A_ub = weights.T
    ones = np.identity((weights.T).shape[0])
    zeros = np.zeros(((weights.T).shape[0],(weights.T).shape[0]))
    #A_ub = np.hstack((A_ub,ones,zeros))
    b_ub = -np.log(1/interval[1,:] - 1) - bias
    b_ub1 = -np.log(1/interval[0,:]-1) - bias
    ones = -np.identity((weights.T).shape[0])
    #A_ub1 = np.hstack((weights.T,zeros,ones))
    A_ub = np.vstack((A_ub,-A_ub))
    b_ub = np.hstack((b_ub,-b_ub1))
    new_interval = np.zeros((A_ub.shape[1]))
    bnds = []
    for j in range(A_ub.shape[1]):
        bnds.append((0,1))
    bnds = tuple(bnds)
    for i in range(A_ub.shape[1]):
        c = np.zeros((A_ub.shape[1]))
        if(mode=="min"):
            c[i] = 1
        else:
            c[i] = -1
        res = scipy.optimize.linprog(c,A_ub = A_ub,b_ub = b_ub,options=dict(tol=1e-8),bounds = bnds)
        #print(res)
        new_interval[i] = res['x'][i]
    return new_interval

In [852]:
import scipy
def backward():
    max_bounds = bound(out_w,out_interval,out_b,"max")
    h1_interval[1,:] = max_bounds
    min_bounds = bound(out_w,out_interval,out_b,"min")
    h1_interval[0,:] = min_bounds
    #max_bounds = bound(h2,h2_interval,b2,"max")
    #min_bounds = bound(h2,h2_interval,b2,"min")
    max_bounds = bound(h1,h1_interval,b1,"max")
    input_interval[1,:] = max_bounds
    min_bounds = bound(h1,h1_interval,b1,"min")
    input_interval[0,:] = min_bounds

In [853]:
backward()

In [854]:
input_interval

array([[0., 0.],
       [1., 1.]])

In [855]:
out_interval

array([[0.13674724],
       [0.40090991]])